## FBA simulation
Simulating the FBA part of the diauxic growth model with `cobrapy` to check that the FBA part is working.
This requires
* `cobrapy >= 0.6.0a3`

In [1]:
from __future__ import print_function, absolute_import

import pandas as pd
import cobra
from matplotlib import pyplot as plt

from sbmlutils import fbc
from sbmlutils.dfba.analysis import set_matplotlib_parameters
from sbmlutils.dfba.diauxic_growth import model_factory

set_matplotlib_parameters()

/usr/local/lib/python2.7/dist-packages/optlang/gurobi_interface.py:26: UserWarning: Be careful! The GUROBI interface is still under construction ...
  warn("Be careful! The GUROBI interface is still under construction ...")


In [2]:
# load model
sbml_path = './results/v{}/diauxic_fba.xml'.format(model_factory.version)
print(sbml_path)
mfba = cobra.io.read_sbml_model(sbml_path)
cobra.io.sbml3.validate_sbml_model(sbml_path)

./results/v9/diauxic_fba.xml


(<Model diauxic_fba at 0x7ff392c0fe50>,
 {'SBML errors': [], 'other': [], 'validator': [], 'warnings': []})

In [3]:
# objective function & boundaries
# pprint(mfba.objective)
fbc.cobra_reaction_info(mfba)

,lb,ub,reversibility,boundary,objective_coefficient,forward_variable,reverse_variable
v1,0,1000,False,False,1,0.0 <= v1 <= 1000.0,0 <= v1_reverse_6654c <= 0
v2,0,1000,False,False,1,0.0 <= v2 <= 1000.0,0 <= v2_reverse_1b267 <= 0
v3,0,1000,False,False,1,0.0 <= v3 <= 1000.0,0 <= v3_reverse_43a03 <= 0
v4,0,1000,False,False,1,0.0 <= v4 <= 1000.0,0 <= v4_reverse_5ed3a <= 0
EX_Ac,-1000,1000,True,True,0,0 <= EX_Ac <= 1000.0,0 <= EX_Ac_reverse_3765e <= 1000.0
EX_Glcxt,-1000,0,False,True,0,0 <= EX_Glcxt <= 0.0,0 <= EX_Glcxt_reverse_34a63 <= 1000.0
EX_O2,-1000,0,False,True,0,0 <= EX_O2 <= 0.0,0 <= EX_O2_reverse_23804 <= 1000.0
EX_X,-1000,0,False,True,0,0 <= EX_X <= 0.0,0 <= EX_X_reverse_ff29b <= 1000.0


In [11]:
# lookat the v4 property
print(mfba.reactions.EX_Ac.reaction)
print(mfba.metabolites.Ac.summary())
# simulate
# s = mfba.optimize()
s = mfba.optimize(objective_sense="maximize")
print(s.status)
mfba.summary()

Ac <=> 
PRODUCING REACTIONS -- acetate (Ac)
-----------------------------------
%    FLUX    RXN ID    REACTION
---  ------  --------  ----------


CONSUMING REACTIONS -- acetate (Ac)
-----------------------------------
%    FLUX    RXN ID    REACTION
---  ------  --------  ----------

None
optimal
IN FLUXES    OUT FLUXES    OBJECTIVES
-----------  ------------  ------------
                           v1  0
                           v2  0
                           v3  0
                           v4  0


In [5]:
# summary of solution
mfba.summary(fva=True)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [ ]:
# FVA
# running flux variability analysis with the model
fva_result = cobra.flux_analysis.flux_variability_analysis(mfba, mfba.reactions)
pd.DataFrame.from_dict(fva_result).T.round(5)

## Change the bounds

In [ ]:
import numpy as np
import pandas as pd
glc_bounds = np.linspace(-1000, 0, 20)
results = []
for lb_glc in glc_bounds:
    mfba.reactions.EX_Glcxt.lower_bound = lb_glc
    s = mfba.optimize(objective_sense="maximize")
    fluxes = s.fluxes
    results.append(fluxes)
df = pd.DataFrame(results)
print(df)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))

for vid in ['v1', 'v2', 'v3', 'v4']:
    ax1.plot(df.EX_Glcxt, df[vid], marker='s', alpha=0.8, label=vid)

ax2.plot([np.min(glc_bounds), np.max(glc_bounds)], [0, 0], color='k', linestyle='-', linewidth=1)
for vid in ['EX_Ac', 'EX_O2', 'EX_Glcxt', 'EX_X']:
    ax2.plot(df.EX_Glcxt, df[vid], marker='s', alpha=0.8, label=vid)
ax2.set_ylim(-1000, 1000)
    
for ax in [ax1, ax2]:
    ax.set_xlabel('Glc species bound')
    ax.legend()

plt.show()